# Creating a custom dataset class to get all csv files from folder

In [167]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import glob
import pandas as pd
import numpy as np
import torch

In [313]:
class StocksDataSet(Dataset):
    def __init__(self, root_directory):
        self.root = root_directory
        self.stocks_files = glob.glob(f"{self.root}/**/*.csv", recursive=True)
        #self.transform = transforms.Compose([transforms.ToTensor()])
        # for i in self.stocks_files:
        #     print(i)
      

    def __len__(self):
        return len(self.stocks_files)

   

    # def create_dataset(input, output, period_in, period_out):
    
    #     X, y = [], []
    #     arr = np.array(input)
    #     arr_two = np.array(output)
    #     for i in range(len(arr) - period_in):
    #         X.append(arr[i:i+period_in])
    #         y.append(arr_two[i + 1 : i + period_out + 1])
    #     #print(y)
    #     #return torch.tensor(test.values).unsqueeze(0), torch.tensor(column[:-period].values).unsqueeze(0)
    #     #return torch.tensor(np.array(X)), torch.tensor(np.array(y))
    #     return np.array(X), np.array(y)
    def create_dataset(self, single_stock, period_in, period_out):
        input = single_stock.drop("Close", axis = 1)
        output = single_stock["Close"]
        X, y = [], []
        arr = np.array(input)
        arr_two = np.array(output)
        for i in range(len(arr) - period_in):
            X.append(arr[i:i+period_in])
            y.append(arr_two[i + 1 : i + period_out + 1])
        #print(y)
        #return torch.tensor(test.values).unsqueeze(0), torch.tensor(column[:-period].values).unsqueeze(0)
        #return torch.tensor(np.array(X)), torch.tensor(np.array(y))
    def test_func(self, single_stock, steps_for_input, steps_for_output):
        input_data = single_stock.drop("Close", axis = 1)
        output_data = single_stock["Close"]
        X,y = [], []
        for i in range(len(input_data)):
            end_x = i + steps_for_input
           # print(end_x)
            output_x = end_x + steps_for_output - 1
            if output_x > len(input_data): 
                break
            #print(output_data[end_x - 1 : output_x])
            seq_x, seq_y = input_data[i : end_x], output_data[end_x - 1 : output_x]
            X.append(seq_x), y.append(seq_y)
            #print(seq_x)
            #print(seq_y)
        print(X)
        return np.array(X), np.array(y)
        
        #return np.array(X), np.array(y)

    def __getitem__(self, stock_index):
      
        data = pd.read_csv(self.stocks_files[stock_index])
        print(data)
       # data.drop("Date", axis = 1, inplace= True)
        
     ### WORKING!!! ###
        # data["Date"] = pd.to_datetime(data["Date"], errors="coerce").dt.tz_localize(None)
        # # data["Date"] = data["Date"].astype('float32')
        # for i in range(len(data)):
        #     data.loc[i, "Date"] = data['Date'][i].timestamp()
        # data["Date"] = data["Date"].astype('float32')
        data["Date"] = (data["Date"].astype('datetime64[ns]')).apply(lambda x : x.timestamp())
        data["Date"] = data["Date"].astype('float64')
        return torch.tensor(data.values)
      #  return self.test_func(data, len(data), 14)
        

In [315]:
# Testing
test = StocksDataSet("../Machine Learning Data Gathering/Stocks_data")

gen = DataLoader(test)
for i in gen:
    print(j)

            Date  Open   High    Low  Close   Volume
0     2016-02-24  5.25  5.250  5.250   5.25      200
1     2016-02-25  5.25  5.250  5.250   5.25      167
2     2016-02-26  6.15  6.150  6.150   6.15      233
3     2016-02-29  6.45  6.450  6.450   6.45      267
4     2016-03-01  6.45  6.450  6.450   6.45        0
...          ...   ...    ...    ...    ...      ...
2010  2024-02-20  2.00  2.005  1.850   1.96   987400
2011  2024-02-21  1.96  2.005  1.850   1.91   507500
2012  2024-02-22  1.91  2.320  1.880   2.27  1938700
2013  2024-02-23  2.25  2.250  2.145   2.18   589000
2014  2024-02-26  2.15  2.350  2.150   2.24   488300

[2015 rows x 6 columns]
tensor([], size=(1, 0), dtype=torch.float64)
           Date        Open        High         Low       Close    Volume
0    2020-12-10  146.000000  165.000000  141.250000  144.710007  70447500
1    2020-12-11  146.550003  151.500000  135.100006  139.250000  26980800
2    2020-12-14  135.000000  135.300003  125.160004  130.000000  1696610